# Wrangle: [testfit.io](https://blog.testfit.io/)

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import wrangle

In [2]:
pd.set_option("display.max_columns", None)
plt.rc("figure", figsize=(16, 8))

## Stakeholder Notes
- Work with Commitments
- Activity Descriptions == New Construction, SubRehab, Additions, and Improvements
- Loan amount >= $1M:
    - Mortgage at time of Firm Commitment Issuance, Amendment or Reissuance?
    - Mortgage at Firm Commitment Issuance?
    - Final Mortgage Amount? **Probably this one...**
- Market == Large urban centers; meaning **cities** not just states; not rural one-off projects
- Types of FHA loan is irrelevant

In [6]:
df = wrangle.wrangle_hud()

AttributeError: module 'wrangle' has no attribute 'wrangle_hud'

In [4]:
print(df.shape)
df.head()

(20918, 29)


,fha_number,project_name,project_city,project_state,"basic_fha,_risk_share_or_other",program_category,activity_description,activity_group,facility_type,program_designation,firm_commitment_activity,lender_at_firm_commitment_activity,"mortgage_at_time_of_firm_commitment_issuance,_amendment_or_reissuance",unit_or_bed_count,date_of_firm_commitment_activity,fiscal_year_of_firm_commitment_activity,mortgage_at_firm_commitment_issuance,date_of_firm_issue,fiscal_year_of_firm_commitment,map_or_tap,lihtc_designation,tax_exempt_bond_designation,home_designation,cdbg_designation,section_202_refi_designation,irp_decoupling_designation,hope_vi_designation,current_status,final_mortgage_amount
0,11345004,Western Heights Apartments,Olney,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,277600,48,2005-10-01,2006,277600,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Firm Comit then Owner Withdrew,271700
1,11345005,Woodlands Village Apts,Breckenridge,TX,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/236,Firm Issued,GREYSTONE SERVICING COMPANY LLC,173000,88,2005-10-01,2006,173000,2005-10-01,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,168300
2,4235583,Harbor Court Apartments,Rocky River,OH,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,GREYSTONE SERVICING COMPANY LLC,3886800,120,2005-10-03,2006,3886800,2005-10-03,2006,TAP,False,False,False,False,False,False,False,Finally Endorsed,3886800
3,3297260,Compton Towne Apartments,Wilmington,DE,Risk Sharing,QPE Risk Sharing,Refinance,Refi/ Pchse,Coop/Apts/Mobile Home Pk/ALF,QPE Risk Shg Existing,Firm Issued,FEDERAL NATIONAL MORTGAGE,4000000,76,2005-10-03,2006,4000000,2005-10-03,2006,TAP,True,False,False,False,False,False,False,Finally Endorsed,4000000
4,7135779,Evergreen Terrace I,Joliet,IL,Basic FHA,223a7 Refi Apts,Refinance,Refi/ Pchse,Apts,a7/221d4,Firm Issued,REILLY MORTGAGE GRP,5129000,241,2005-10-04,2006,5129000,2005-10-04,2006,TAP,False,False,False,False,False,False,False,Firm Expired,5247700


AttributeError: 'DataFrame' object has no attribute 'mortgage_at_time_of_firm_commitment_issuance'

In [ ]:
# commitments
def acquire_fha_data():
    """take fha data downloaded into csv and turn it into pandas df"""
    url = 'https://www.hud.gov/sites/dfiles/Housing/documents/Initi_Endores_Firm%20Comm_DB_FY06_FY20_Q2.xlsx'
    xlsx = "Initi_Endores_Firm Comm_DB_FY06_FY20_Q2.xlsx"
    sheet_name = "Firm Cmtmts, Iss'd and Reiss'd"
    df  = pd.read_excel(url, sheet_name=sheet_name, header=6)
    return df


In [ ]:
df = acquire_fha_data()

In [ ]:
df.shape

In [ ]:
df.to_csv("clean_data.csv")

In [ ]:
def snake_case_column_names(df):
    """takes in a data frame, lower cases column name, replaces space with underscore and strips commas"""
    new_column_names = []
    
    for col in df.columns:
        new_column_names.append(col.lower().strip(',').replace(" ", "_"))
    
    df.columns = new_column_names
    return df

In [ ]:
df = snake_case_column_names(df)

In [ ]:
def change_to_bool(df):
    """takes in columns with only 2 options and turns them into booleans"""
    for col in df.columns:
        if (df[col][0] == 0) & (df[col].unique()[1] == 'Y'):
            df[col] = df[col] == "Y"
    return df

In [ ]:
df = change_to_bool(df)

In [ ]:
df.info()

In [ ]:
# change fha number to object
# change fiscal year of firm commitment activity to datetime year only
# change fiscal year of firm commitment to datetime year only
def changing_data_types(df):
    """change fha number to an object
    change year only columns to datetime"""

    df.fiscal_year_of_firm_commitment = pd.to_datetime(df.fiscal_year_of_firm_commitment, format="%Y")
    df.fiscal_year_of_firm_commitment = df.fiscal_year_of_firm_commitment.apply(lambda x: x.year)

    df.fiscal_year_of_firm_commitment_activity = pd.to_datetime(df.fiscal_year_of_firm_commitment_activity, format="%Y")
    df.fiscal_year_of_firm_commitment_activity = df.fiscal_year_of_firm_commitment_activity.apply(lambda x: x.year)

    df.fha_number = df.fha_number.astype('object')
    return df

In [ ]:
milestone = pd.to_datetime("Feb 4 2019")
milestone.year

In [ ]:
# activity_groups
# df = df[(df.activity_group == "NC/SR") | (df.activity_group == "Imp/Adds")]

In [ ]:
# df = df[df["Final Mortgage Amount"] >= 1000000]
# df.head()

In [ ]:
df.info()

In [ ]:
df["Activity Description"].value_counts()

In [ ]:
df["activity_group"].value_counts()

In [ ]:
df.rename(columns={"Facility Type": "facility_type"}, inplace=True)

In [ ]:
df.facility_type.value_counts()

In [ ]:
df.rename(columns={"FHA Number": "fha_number"}, inplace=True)

In [ ]:
df.groupby(["Project State"]).fha_number.count().sort_values().plot.barh()
plt.show()

In [ ]:
df[df["Final Mortgage Amount"] >= 1000000].shape

In [ ]:
df.rename(columns={"Final Mortgage Amount": "final_mortgage"}, inplace=True)

In [ ]:
df.groupby(["Project State"]).final_mortgage.sum().sort_values().plot.barh()

In [ ]:
df["Project City"].value_counts().head(20)

In [ ]:
df.groupby(["Project City", "Fiscal Year of Firm Commitment"]).size().sort_values()

In [ ]:
df.groupby(["Project City", "Fiscal Year of Firm Commitment"])["final_mortgage"].sum().sort_values()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["final_mortgage"].sum()

In [ ]:
df[df["Project City"] == "Minneapolis"].groupby("Fiscal Year of Firm Commitment")["final_mortgage"].sum()

In [ ]:
# csv works and can be read
pd.read_csv("hud_commitments.csv", index_col=0)

In [ ]:
df.activity_group.value_counts()

In [ ]:
san_antonio = df[df['Project City'] == "San Antonio"]

In [ ]:
san_antonio[san_antonio['Activity Description'] == 'New Construction']

In [ ]:
for col in df.columns:
    unique = len(df[col].unique())
    if unique > 10:
        print()
        print(f'{col} has {unique} values')
    else:
        print()
        print(f'{col} value count:')
        print(df[col].value_counts())
        

In [ ]:
df.head()

In [ ]:
df